In [155]:

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os 

from sklearn.linear_model import LinearRegression 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [156]:
# Conectando com o db
data_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'data', 'data_tratados.parquet'))
# Retornando dado sem um df 
df = pd.read_parquet(data_path)
df.sample(5)

,marca,modelo,uf,vidros_eletricos,ar_condicionado,valor,VALOR_LOG,KM,KM_LOG,ano,...,híbrido,outros,álcool,Assistida,Elétrica,Hidráulica,Mecânica,Automática,Manual,Semiautomática
7305,citroen,C3 1.6 EXCLUSIVE 16V FLEX 4P AUTOMÁTICO,RS,1,1,38000.0,10.545341,12930.0,9.467305,2013.0,...,0,0,0,0,0,1,0,1,0,0
540,volkswagen,Virtus Mf,PR,1,1,78900.0,11.275937,10810.0,9.288227,2020.0,...,0,0,0,0,1,0,0,0,1,0
8946,peugeot,208 Allure 1.6 (Flex) (Aut),MT,1,1,78580.0,11.271872,5750.0,8.656955,2023.0,...,0,0,0,0,1,0,0,1,0,0
772,volkswagen,Tiguan 2.0 350 TSI GASOLINA ALLSPACE R-LINE 4M...,SP,1,1,218990.0,12.296781,6292.0,8.747034,2019.0,...,0,0,0,0,1,0,0,1,0,0
967,volkswagen,Jetta Highline Tiptronic 2.0 Tsi Aut.,DF,1,1,99900.0,11.511925,12000.0,9.392662,2015.0,...,0,0,0,0,0,1,0,1,0,0


## Modelagem

### Separando base de treino e teste


In [37]:
df.columns

Index(['marca', 'modelo', 'uf', 'vidros_eletricos', 'ar_condicionado', 'valor',
       'VALOR_LOG', 'KM', 'KM_LOG', 'ano', 'ANO_LOG', 'diesel', 'elétrico',
       'flex', 'gasolina', 'gasolina e gás natural', 'híbrido', 'outros',
       'álcool', 'Assistida', 'Elétrica', 'Hidráulica', 'Mecânica'],
      dtype='object')

In [154]:
dados_modelo

,vidros_eletricos,ar_condicionado,VALOR_LOG,KM,ano,diesel,elétrico,flex,gasolina,gasolina e gás natural,híbrido,outros,álcool,Assistida,Elétrica,Hidráulica,Mecânica
0,1,1,10.987832,5436.0,2023.0,0,0,1,0,0,0,0,0,0,0,0,1
1,1,1,10.940224,3980.0,2023.0,0,0,1,0,0,0,0,0,0,0,0,1
2,1,1,11.632485,3857.0,2020.0,0,0,1,0,0,0,0,0,0,1,0,0
3,1,1,10.989015,4900.0,2023.0,0,0,1,0,0,0,0,0,0,0,0,1
4,1,1,10.817776,8210.0,2017.0,0,0,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10464,0,1,12.788288,3800.0,2022.0,0,0,0,0,0,1,0,0,0,1,0,0
10465,1,1,11.643076,2500.0,2019.0,0,0,0,1,0,0,0,0,0,1,0,0
10466,0,1,12.206068,5054.0,2021.0,0,0,0,0,0,1,0,0,0,1,0,0
10467,0,1,11.982304,6140.0,2019.0,0,0,0,1,0,0,0,0,0,1,0,0


In [231]:
## Lista com colunas que não serão usadas
drop_colunas_x = ['KM','ANO_LOG','valor', 'marca', 'modelo', 'uf']

dados_modelo = df.copy()
dados_modelo = dados_modelo.drop(columns=drop_colunas_x)

# Separa os dados em features (X) e target (y)
X_valores_abs = dados_modelo.drop('VALOR_LOG', axis=1)
y_valores_abs = dados_modelo['VALOR_LOG']



# Dividi os dados em treino e teste para valores orginais
X_treino_abs, X_teste_abs, y_treino_abs, y_teste_abs = train_test_split(X_valores_abs, y_valores_abs, test_size=0.2, random_state=1234)
# # Dividi os dados em treino e teste para valores LOG
# X_treino_abs, X_teste_abs, y_treino_abs, y_teste_abs = train_test_split(X_valores_abs, y_valores_abs, test_size=.2, random_state=1234)

### Treinando Modelos

* LinearRegression

In [232]:
resultados = []
lr = LinearRegression()
lr.fit(X_treino_abs,y_treino_abs)
resultados.append( ('lr',lr.score(X_teste_abs,y_teste_abs)) )

print("Métrica R²")
print(lr.score(X_teste_abs,y_teste_abs))

Métrica R²
0.7172203685706404


* DecisionTreeRegressor

In [233]:
dtr = DecisionTreeRegressor()
dtr.fit(X_treino_abs,y_treino_abs)
resultados.append( ('dtr', dtr.score(X_teste_abs,y_teste_abs)) )
print("Métrica R²")
print(dtr.score(X_teste_abs,y_teste_abs))

Métrica R²
0.5323633752669198


* RandomForestRegressor

In [234]:
rfr = RandomForestRegressor()
rfr.fit(X_treino_abs,y_treino_abs)
resultados.append( ('rfr', rfr.score(X_teste_abs,y_teste_abs)) )
print("Métrica R²")
print(dtr.score(X_teste_abs,y_teste_abs))

Métrica R²
0.5323633752669198


* Validando Modelo

In [235]:
resultados

[('lr', 0.7172203685706404),
 ('dtr', 0.5323633752669198),
 ('rfr', 0.6938929465687957)]